# Classifying documents using a bag-of-words approach

In [31]:
import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [11]:
logging.basicConfig(level=logging.INFO, format='%(asctime) %(levelname) %(message)s')

op = OptionParser()
op.add_option('--report', action="store_true", dest="print_report", help="Print a detailed classification report.")
op.add_option('--chi2_select', action="store", type="int", dest="select_chi2", help="Select some number of features using chi-squared test")
op.add_option('--confusion matrix', action="store_true", dest="print_cm", help="Print the confusion matrix")
op.add_option("--top10", action="store_true", dest="print_top10", help="Print ten most discriminative terms per class for every classifier")
op.add_option("--all_categories", action="store_true", dest="all_categories", help="Whether to use all categories or not")
op.add_option("--use_hashing", action="store_true", help="Use hashing vectorizer")
op.add_option("--n_features", action="store", type="int", help="n_featurs when using hashing vectorizer.")
op.add_option("--filtered", action="store_true", help="Remove newsgroup information that is easily overfit: headers, signatures, and quoting")

def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)
print(__doc__)
op.print_help()
print()

Automatically created module for IPython interactive environment
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --report              Print a detailed classification report.
  --chi2_select=SELECT_CHI2
                        Select some number of features using chi-squared test
  --confusion matrix    Print the confusion matrix
  --top10               Print ten most discriminative terms per class for
                        every classifier
  --all_categories      Whether to use all categories or not
  --use_hashing         Use hashing vectorizer
  --n_features=N_FEATURES
                        n_featurs when using hashing vectorizer.
  --filtered            Remove newsgroup information that is easily overfit:
                        headers, signatures, and quoting



## Load data from the training set
Load data from the newsgroup dataset which comprises around 18000 newsgroups posts on 20 topics split in two subsets, one for training and the other for testing

In [3]:
if opts.all_categories:
    categories = None
else:
    categories = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space'
    ]
    
if opts.filtered:
    remove = ('headers', 'footers', 'quotes')
else:
    remove = ()
    
print("Loading 20 newsgroups dataset for categories:")
print(categories if categories else "all")

data_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, remove=remove)
data_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, remove=remove)

print('data loaded')

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
data loaded


In [19]:
target_names = data_train.target_names

def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

data_train_size_mb = size_mb(data_train.data)
data_test_size_mb = size_mb(data_test.data)

print('%d documents - %0.3fMB (training set)' % (len(data_train.data), data_train_size_mb))
print('%d documents - %0.3fMB (testing set)' % (len(data_test.data), data_test_size_mb))
print("%d categories " % len(target_names))
print()

y_train, y_test = data_train.target, data_test.target

print("Extracting features from the training data using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False, n_features=opts.n_featurs)
    X_train = vectorizer.transform(data_train.data)
else:
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
    X_train = vectorizer.fit_transform(data_train.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_train.shape)
print()


print("Extracting features from the test data using the same vectorizer")
t0 = time()
X_test = vectorizer.transform(data_test.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_test.shape)
print()


2034 documents - 3.980MB (training set)
1353 documents - 2.867MB (testing set)
4 categories 

Extracting features from the training data using a sparse vectorizer
done in 0.396706s at 10.031MB/s
n_samples: 2034, n_features: 33809

Extracting features from the test data using the same vectorizer
done in 0.236674s at 12.116MB/s
n_samples: 1353, n_features: 33809



In [12]:
# mapping from integer feature name to original token string
if opts.use_hashing:
    feature_names = None
else:
    feature_names = vectorizer.get_feature_names()
    
if opts.select_chi2:
    print("Extracting %d best features by a chi-squared test " %opt.select_chi2)
    t0 = time()
    ch2 = SelectGest(chi2, k=opts.select_chi2)
    X_train = ch2.fir_transform(X_train, y_train)
    X_test = ch2.transform(X_test)
    if feature_names:
        # kee selected feature names
        feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]
    print("Done in %fs" % time() - t0)
    print()
    
if feature_names:
    feature_names = np.asarray(feature_names)
    
def trim(s):
    return s if len(s) <= 80 else s[:77] + "..."

## Benchmark Classifiers
We train classifiers with 15 different classification models and get performance results for each model

In [37]:
def benchmark(clf):
    print("_"*80)
    print("Training")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("Train time: %0.3fs" % train_time)
    
    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time: %0.3fs" %test_time)
    
    score = metrics.accuracy_score(y_test, pred)
    print("accuracy: %0.3f" % score)
    
    if hasattr(clf, "coef_"):
        print("Dimensionality: %d" % clf.coef_.shape[1])
        print("Density: %f" %density(clf.coef_))
        
        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
            print()
        if opts.print_report:
            print("classification report:")
            print(metrics.classification_report(y_test, pred, target_names=target_names))
        
        if opts.print_cm:
            print("confusion matrix")
            print(metric.confusion_matrix(y_test, pred))
        
        print()
        clf_descr = str(clf).split('(')[0]
        return clf_descr, score, train_time, test_time
    
results = []
for clf, name in (
    (linear_model.RidgeClassifier(tol=1e-2, solver="sag"), "Ridge Classifier"),
    (linear_model.Perceptron(max_iter=50), "Perceptron"),
    (linear_model.PassiveAggressiveClassifier(max_iter=50), "Passive-Aggressive"),
    (KNeighborsClassifier(n_neighbors=10), "kNN"),
    (RandomForestClassifier(), "Random Forest")):
    print("="*80)
    print(name)
    results.append(benchmark(clf))
    
for penalty in ["l1", "l2"]:
    print("="*80)
    print("%s penalty" % penalty.upper())
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False, tol=1e-3)))
    results.append(benchmark(linear_model.SGDClassifier(alpha=0.0001, max_iter=50, penalty=penalty)))
    
# train SGD with elastic net penalty
print("="*80)
print("Nearestcentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# train sparse naive bayes classifiers
print("="*80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=0.01)))
results.append(benchmark(BernoulliNB(alpha=0.1)))
results.append(benchmark(ComplementNB(alpha=.1)))

print("="*80)
print("LinearSVC with L1-based feature selection")
# the smaller the C, the stronger the regularization
# the more regularization, the more the sparsity
results.append(benchmark(Pipeline([
    ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False, tol=1e-3))),
    ('classification', LinearSVC(penalty="l2"))
])))

Ridge Classifier
________________________________________________________________________________
Training
RidgeClassifier(solver='sag', tol=0.01)
Train time: 0.175s
test time: 0.002s
accuracy: 0.897
Dimensionality: 33809
Density: 1.000000

Perceptron
________________________________________________________________________________
Training
Perceptron(max_iter=50)


/home/duke/duke/programming/advancedpython/venv/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:555: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


Train time: 0.037s
test time: 0.004s
accuracy: 0.888
Dimensionality: 33809
Density: 0.255302

Passive-Aggressive
________________________________________________________________________________
Training
PassiveAggressiveClassifier(max_iter=50)
Train time: 0.069s
test time: 0.004s
accuracy: 0.902
Dimensionality: 33809
Density: 0.701611

kNN
________________________________________________________________________________
Training
KNeighborsClassifier(n_neighbors=10)
Train time: 0.002s
test time: 0.234s
accuracy: 0.858
Random Forest
________________________________________________________________________________
Training
RandomForestClassifier()
Train time: 1.347s
test time: 0.076s
accuracy: 0.838
L1 penalty
________________________________________________________________________________
Training
LinearSVC(dual=False, penalty='l1', tol=0.001)
Train time: 0.194s
test time: 0.001s
accuracy: 0.873
Dimensionality: 33809
Density: 0.005553

______________________________________________________

In [41]:
# Add Plots
indices = np.arange(len(results))

results = [[x[i] for x in results] for i in range(4)]
clf_names, score, training_time, test_time = results
training_time = np.array(training_time) / np.max(training_time)
test_time = np.array(test_time) / np.max(test_time)

pt.figure(figsize=(12, 8))
plt.title("Score")
plt.barh(indices, score, .2, label="score", color="navy")
plt.barh(indices + .3, training_time, .2, label="training_time", color="c")
plt.barh(indices + .6, test_time, .2, label="test time", color="darkorange")
plt.yticks()
plt.legend(loc="best")
plt.subplots_adjust(left=.25)
plt.subplots_adjust(top=.95)
plt.subplots_adust(bottom=.05)

for i, c in zip(indices, clf_names):
    plt.text(-.3, i, c)
    
plt.show()

[('RidgeClassifier',
  0.8965262379896526,
  0.17455697059631348,
  0.0016829967498779297),
 ('Perceptron', 0.8876570583887657, 0.03737020492553711, 0.003830432891845703),
 ('PassiveAggressiveClassifier',
  0.9024390243902439,
  0.0685570240020752,
  0.004080533981323242),
 None,
 None,
 ('LinearSVC', 0.8728750923872876, 0.193772554397583, 0.001447916030883789),
 ('SGDClassifier',
  0.8832224685883222,
  0.2257215976715088,
  0.003748655319213867),
 ('LinearSVC', 0.9002217294900222, 0.4277362823486328, 0.002930164337158203),
 ('SGDClassifier',
  0.9024390243902439,
  0.06769657135009766,
  0.003930330276489258),
 None,
 ('MultinomialNB',
  0.8994826311899483,
  0.009933948516845703,
  0.003675699234008789),
 ('BernoulliNB',
  0.877309682187731,
  0.011932134628295898,
  0.011772871017456055),
 ('ComplementNB',
  0.9105691056910569,
  0.010434389114379883,
  0.0036454200744628906),
 None]